<a href="https://colab.research.google.com/github/yashgupta-7/legendre-forecasters/blob/main/da_rnn/darnn_lmu_keras_logsparse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 !git clone https://github.com/kaelzhang/DA-RNN-in-Tensorflow-2-and-PyTorch/

Cloning into 'DA-RNN-in-Tensorflow-2-and-PyTorch'...
remote: Enumerating objects: 256, done.
remote: Counting objects: 100% (256/256), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 256 (delta 127), reused 184 (delta 60), pack-reused 0
Receiving objects: 100% (256/256), 7.64 MiB | 19.50 MiB/s, done.
Resolving deltas: 100% (127/127), done.


In [2]:
!git clone https://github.com/nengo/keras-lmu

Cloning into 'keras-lmu'...
remote: Enumerating objects: 738, done.
remote: Counting objects: 100% (228/228), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 738 (delta 130), reused 162 (delta 73), pack-reused 510
Receiving objects: 100% (738/738), 10.49 MiB | 13.51 MiB/s, done.
Resolving deltas: 100% (377/377), done.


In [3]:
# import keras_lmu

In [4]:
!pwd

/content


In [5]:
import numpy as np
import tensorflow as tf
import random as python_random
import math
#final
np.random.seed(123)
python_random.seed(123)
tf.random.set_seed(1234)

In [6]:
cd /content/DA-RNN-in-Tensorflow-2-and-PyTorch/

/content/DA-RNN-in-Tensorflow-2-and-PyTorch


In [7]:
# !pip install keras_lmu
!pip install get_rolling_window
!ls /content/keras-lmu/keras_lmu/

__init__.py  layers.py	tests  version.py


In [8]:
from typing import Optional
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import (
    Layer,
    LSTM,
    Dense,
    Permute
)
from tensorflow.keras.models import Model
from da_rnn.common import (
    check_T
)

import os
os.sys.path.insert(0, "/content/keras-lmu/")
# from sibling import module
import keras_lmu
tf.random.set_seed(
    42
)

In [9]:
class InputAttention(Layer):
    T: int

    def __init__(self, T, **kwargs):

        super().__init__(name='input_attention', **kwargs)

        self.T = T

        self.W_e = Dense(T)
        self.U_e = Dense(T)
        self.v_e = Dense(1)

    def call(
        self,
        hidden_state,
        cell_state,
        X
    ):

        n = X.shape[2]

        # [h_t-1; s_t-1]
        hs = K.repeat(
            tf.concat([hidden_state, cell_state], axis=-1),
            # -> (batch_size, m * 2); (batch_size, m+d) if LMU
            n
        )
        # -> (batch_size, n, m * 2); (batch_size, n, m + d) if LMU

        tanh = tf.math.tanh(
            tf.concat([
                self.W_e(hs),
                # -> (batch_size, n, T)

                self.U_e(
                    Permute((2, 1))(X)
                    # -> (batch_size, n, T)
                ),
                # -> (batch_size, n, T)
            ], axis=-1)
            # -> (batch_size, n, T * 2)
        )
        # -> (batch_size, n, T * 2)

        # Equation 8:
        e = self.v_e(tanh)
        # -> (batch_size, n, 1)

        # Equation: 9
        return tf.nn.softmax(
            Permute((2, 1))(e)
            # -> (batch_size, 1, n)
        )
        # -> (batch_size, 1, n)

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'T': self.T
        })
        return config

In [10]:
# !pip uninstall keras_lmu

In [11]:
import keras_lmu
# import gputil
import tensorflow as tf 
if tf.test.gpu_device_name(): 
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [12]:
 def log_mask( win_len, sub_len):
        mask = np.zeros((win_len, win_len))
        for i in range(win_len):
            mask[i] = row_mask(i, sub_len, win_len)
        return mask

def row_mask( index, sub_len, win_len):
    
    log_l = math.ceil(np.log2(sub_len))
    mask = np.zeros((win_len))
    if((win_len // sub_len) * 2 * (log_l) > index):
        mask[:(index + 1)] = 1
    else:
        while(index >= 0):
            if((index - log_l + 1) < 0):
                mask[:index] = 1
                break
            mask[index - log_l + 1:(index + 1)] = 1  # Local attention
            for i in range(0, log_l):
                new_index = index - log_l + 1 - 2**i
                if((index - new_index) <= sub_len and new_index >= 0):
                    mask[new_index] = 1
            index -= sub_len
    return mask

In [41]:
def get_temporal_mask(win_len,sub_len): #win_len=T
  a=log_mask(win_len,sub_len)
  # print(a)
  b = a+np.flipud(np.fliplr(a))-np.eye(win_len)
  # np.fill_diagonal(b, 1)
  return b #np.transpose(a) 


In [14]:
class Encoder(Layer):
    T: int
    m: int
    d: int

    def __init__(
        self,
        T: int,
        m: int,
        d: int,
        lmu=False,
        **kwargs
    ):

        super().__init__(name='encoder_input', **kwargs)

        self.T = T
        self.m = m
        self.d = d
        self.lmu = lmu
        #n_embed + n_
        if not lmu:
            assert(m==d)

        if self.lmu:
          self.input_lstm = keras_lmu.LMUCell(memory_d=self.d, order=self.d, theta=self.T, #LMU
                hidden_cell = tf.keras.layers.SimpleRNNCell(m), # m -> 	Positive integer, dimensionality of the output space.
                hidden_to_memory=False,
                memory_to_memory=False,
                input_to_hidden=True,
                kernel_initializer="ones",
               #return_sequences = False
            )
        else:
          self.input_lstm = LSTM(m, return_state=True)
        self.input_attention = InputAttention(T)

    def call(self, X) -> tf.Tensor:

        batch_size = K.shape(X)[0]

        hidden_state = tf.zeros((batch_size, self.m))
        if self.lmu:
          # print('lmuuu')
          cell_state = tf.zeros((batch_size, self.d*self.d))
        else:
          cell_state = tf.zeros((batch_size, self.m))

        X_encoded = []

        for t in range(self.T):
            # print(t)
            Alpha_t = self.input_attention(hidden_state, cell_state, X)
            # print('hello prat')
            # print(Alpha_t.shape)
            # print(X[:, None, t, :].shape)
            # Equation 10
            X_tilde_t = tf.multiply(
                Alpha_t,
                # TODO:
                # make sure it can share the underlying data
                X[:, None, t, :]
            )
            # -> (batch_size, 1, n)

            # Equation 11
            # print("Xtilde", X_tilde_t.shape, "Hidden", hidden_state.shape, "Mem", cell_state.shape)
            
            if self.lmu:
              X_tilde_t = X_tilde_t[:, 0, :]
              # print("Xtilde", X_tilde_t.shape)
              # print("Out", self.input_lstm(X_tilde_t,states = [hidden_state, cell_state])[0].shape)
              _ , final_state = self.input_lstm(X_tilde_t,
              states = [hidden_state, cell_state])
              
              hidden_state = final_state[0]
              cell_state = final_state[-1]
            else:
              hidden_state, _, cell_state = self.input_lstm(
                X_tilde_t,
                initial_state=[hidden_state, cell_state]
            )

            X_encoded.append(
                hidden_state[:, None, :]
                # -> (batch_size, 1, m)
            )

        return tf.concat(X_encoded, axis=1)
        # -> (batch_size, T, m)

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'T': self.T,
            'm': self.m,
            'd': self.d
        })
        return config

In [15]:
class TemporalAttention(Layer):
    m: int

    def __init__(self, m: int, **kwargs):

        super().__init__(name='temporal_attention', **kwargs)
        
        self.m = m

        self.W_d = Dense(m)
        self.U_d = Dense(m)
        self.v_d = Dense(1)

    def call(
        self,
        hidden_state,
        cell_state,
        X_encoded
    ):

        # Equation 12
        l = self.v_d(
            tf.math.tanh(
                tf.concat([
                    self.W_d(
                        K.repeat(
                            tf.concat([hidden_state, cell_state], axis=-1),
                            # -> (batch_size, p * 2)
                            X_encoded.shape[1]
                        )
                        # -> (batch_size, T, p * 2)
                    ),
                    # -> (batch_size, T, m)
                    self.U_d(X_encoded)
                ], axis=-1)
                # -> (batch_size, T, m * 2)
            )
            # -> (batch_size, T, m)
        )
        # -> (batch_size, T, 1)

        # Equation 13
        return tf.nn.softmax(l, axis=1)
        # -> (batch_size, T, 1)

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'm': self.m
        })
        return config


In [16]:
a=tf.convert_to_tensor(get_temporal_mask(12,20),dtype=tf.float32)
print(a) #[:,49]

print(tf.multiply(-a[None,:,49:50],a[None,:,49:50]))
print(tf.multiply(-a[None,:,49:50],a[None,:,49:50]).shape)

tf.Tensor(
[[0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0.]
 [1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0.]
 [1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0.]
 [1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.]
 [0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1.]
 [0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1.]
 [0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0.]], shape=(12, 12), dtype=float32)
tf.Tensor([], shape=(1, 12, 0), dtype=float32)
(1, 12, 0)


In [17]:
class Decoder(Layer):
    T: int
    m: int
    p: int
    d: int
    y_dim: int

    def __init__(
        self,
        T: int,
        m: int,
        p: int,
        d: int,
        y_dim: int,
        lmu=False,
         **kwargs
    ):

        super().__init__(name='decoder', **kwargs)
        self.log_mask = tf.convert_to_tensor(get_temporal_mask(T,20),dtype=tf.float32)
        self.T = T
        self.m = m
        self.p = p
        self.d = d
        self.lmu = lmu
        self.y_dim = y_dim

        self.temp_attention = TemporalAttention(m)
        self.dense = Dense(1)
        if not lmu:
            assert(p==d)
        
        if self.lmu:
          # print('hello')
          self.decoder_lstm = keras_lmu.LMUCell(memory_d=self.d, order=self.d, theta=self.T, #LMU
                hidden_cell = tf.keras.layers.SimpleRNNCell(p), # m -> 	Positive integer, dimensionality of the output space.
                hidden_to_memory=False,
                memory_to_memory=False,
                input_to_hidden=True,
                kernel_initializer="ones",
               #return_sequences = False
            )
        else:
          self.decoder_lstm = LSTM(p, return_state=True)

        self.Wb = Dense(p)
        self.vb = Dense(y_dim)

    def call(self, Y, X_encoded) -> tf.Tensor:

        batch_size = K.shape(X_encoded)[0]
        hidden_state = tf.zeros((batch_size, self.p))
        if self.lmu:
          cell_state = tf.zeros((batch_size, self.d*self.d))
        else:
          cell_state = tf.zeros((batch_size, self.p))

        # c in the paper
        context_vector = tf.zeros((batch_size, 1, self.m))
        # -> (batch_size, 1, m)

        for t in range(self.T - 1):
            # print('de'+str(t))
            Beta_t = self.temp_attention(
                hidden_state,
                cell_state,
                X_encoded
            )
            # -> (batch_size, T, 1)

            # Equation 14
            context_vector = tf.matmul(
                #Beta_t, X_encoded, transpose_a=True
                 tf.multiply(Beta_t,self.log_mask[None,:,t:t+1]), X_encoded, transpose_a=True
            )
            # print('hello sp')
            # print(Beta_t.shape,end='**\n')
            # print(X_encoded.shape,end='**\n')
            # -> (batch_size, 1, m)

            # Equation 15
            y_tilde = self.dense(
                tf.concat([Y[:, None, t, :], context_vector], axis=-1)
                # -> (batch_size, 1, y_dim + m)
            )
            # -> (batch_size, 1, 1)

            # Equation 16
            if self.lmu:
              y_tilde = y_tilde[:, 0, :]
              _ , final_state = self.decoder_lstm(y_tilde,
              states = [hidden_state, cell_state])
              
              hidden_state = final_state[0]
              cell_state = final_state[-1]
            else:
              hidden_state, _, cell_state = self.decoder_lstm(
                  y_tilde,
                  initial_state=[hidden_state, cell_state]
              )
            # -> (batch_size, p)

        concatenated = tf.concat(
            [hidden_state[:, None, :], context_vector], axis=-1
        )
        # -> (batch_size, 1, m + p)

        # Equation 22
        y_hat_T = self.vb(
            self.Wb(concatenated)
            # -> (batch_size, 1, p)
        )
        # -> (batch_size, 1, y_dim)
        # print('crax')
        return tf.squeeze(y_hat_T, axis=1)

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'T': self.T,
            'm': self.m,
            'p': self.p,
            'y_dim': self.y_dim
        })
        return config

In [18]:
class DARNN(Model):
    def __init__(
        self,
        T: int,
        m: int,
        d: int,
        p: Optional[int] = None,
        y_dim: int = 1,
        lmu=False
    ):

        super().__init__(name='DARNN')

        check_T(T)

        self.T = T
        self.m = m
        self.d = d
        self.p = p or m
        self.y_dim = y_dim
        self.lmu = lmu

        self.encoder = Encoder(T, m, d=self.d, lmu=self.lmu)
        self.decoder = Decoder(T, m, p, d=self.d, lmu=self.lmu, y_dim=y_dim)

    # Equation 1
    def call(self, inputs):
        # print('here')
        X = inputs[:, :, :-self.y_dim]
        # -> (batch_size, T, n)

        # Y's window size is one less than X's
        # so, abandon `y_T`

        # By doing this, there are some benefits which makes it pretty easy to
        # process datasets
        Y = inputs[:, :, -self.y_dim:]
        # -> (batch_size, T - 1, y_dim)

        X_encoded = self.encoder(X)
        # print('koool')
        y_hat_T = self.decoder(Y, X_encoded)
        # -> (batch_size, y_dim)

        return y_hat_T

    def get_config(self):
        return {
            'T': self.T,
            'm': self.m,
            'p': self.p,
            'd': self.d,
            'y_dim': self.y_dim
        }

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
dataroot = '/content/drive/MyDrive/aml_project/data/nasdaq100_padding.csv'

In [21]:
!ls '/content/drive/MyDrive/aml_project/data/nasdaq100_padding.csv'

/content/drive/MyDrive/aml_project/data/nasdaq100_padding.csv


In [22]:
import pandas as pd


def read_data(input_path, debug=True): 

    df = pd.read_csv(input_path, nrows=32*10 if debug else None)
    X = df.loc[:, [x for x in df.columns.tolist()]].to_numpy() 
    return X

In [23]:
from sklearn.preprocessing import StandardScaler
features = read_data(dataroot, debug=False)
scale = StandardScaler().fit(features)
features = scale.transform(features)

In [24]:
features.shape

(40560, 82)

In [25]:
features

array([[-0.82166581, -3.42991957, -1.60918711, ..., -1.86674079,
        -2.14183171, -2.35353513],
       [-0.83220311, -3.40824969, -1.60918711, ..., -1.81032478,
        -2.06400103, -2.35353513],
       [-0.83792335, -3.3938031 , -1.66989855, ..., -1.70183245,
        -2.05801406, -2.34133626],
       ...,
       [ 1.87717227,  1.16289176,  0.06697655, ...,  3.04774477,
        -1.84344086,  1.7048016 ],
       [ 1.88362367,  1.1737267 ,  0.07753506, ...,  3.0501316 ,
        -1.84547643,  1.72163035],
       [ 1.88362367,  1.17131893,  0.0722558 , ...,  3.05447129,
        -1.84248294,  1.72151974]])

In [26]:
# !pip install get-rolling-window
BATCH_SIZE = 256
WINDOW_SIZE = 10
ENCODER_HIDDEN_STATES = 128
DECODER_HIDDEN_STATES = 128

Y_DIM = 1

DATA = 'nasdaq100_padding.csv'

VALIDATION_RATIO = 0.2

EPOCHS = 30

In [27]:
def get_labels_from_features(features):
    return features[WINDOW_SIZE - 1:, -Y_DIM:][:, None, :]


def split_by_ratio(features):
    length = len(features)
    train_length = 35100
    validation_length = 2730 #int(VALIDATION_RATIO * length)
    test_length = 2730
    
    return features[:train_length], features[train_length:train_length + validation_length], features[train_length + validation_length:train_length + validation_length + test_length] #features[:-validation_length], features[-validation_length:]


training_features, validation_features, testing_features = split_by_ratio(features)

print('training length', len(training_features))
print('validation length', len(validation_features))
print('testing length', len(testing_features))

training length 35100
validation length 2730
testing length 2730


In [28]:
import tensorflow as tf
from get_rolling_window import rolling_window

train_f, train_l = rolling_window(training_features, WINDOW_SIZE, 1), get_labels_from_features(training_features)
train_ds = tf.data.Dataset.from_tensor_slices((train_f, train_l)).batch(BATCH_SIZE, drop_remainder=False)

print(train_f.shape, train_l.shape)

val_f, val_l = rolling_window(validation_features, WINDOW_SIZE, 1), get_labels_from_features(validation_features)
val_ds = tf.data.Dataset.from_tensor_slices((val_f, val_l)).batch(BATCH_SIZE, drop_remainder=False)

test_f, test_l = rolling_window(testing_features, WINDOW_SIZE, 1), get_labels_from_features(testing_features)
test_ds = tf.data.Dataset.from_tensor_slices((test_f, test_l)).batch(BATCH_SIZE, drop_remainder=False)

train_ds

(35051, 50, 82) (35051, 1, 1)


<BatchDataset shapes: ((None, 50, 82), (None, 1, 1)), types: (tf.float64, tf.float64)>

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Reshape

model = DARNN(
    T = WINDOW_SIZE,
    m = ENCODER_HIDDEN_STATES,
    d = ENCODER_HIDDEN_STATES,
    p = DECODER_HIDDEN_STATES,
    y_dim = Y_DIM,
    lmu = False,
    
)

model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae', 'mape']
)

In [ ]:
feature_batch, label_batch = next(iter(train_ds))

print('feature, label shape', feature_batch.shape, label_batch.shape)

print('prediction shape', model(feature_batch).shape)

model(feature_batch[:1])


feature, label shape (256, 50, 82) (256, 1, 1)
prediction shape (256, 1)


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.00524321]], dtype=float32)>

In [ ]:
save_to = './ckpt_darnn_lstm_logsparse.hdf5'

history = model.fit(
    train_ds,
    validation_data=val_ds,
    callbacks=[
        # Save checkpoints on best validation loss
        tf.keras.callbacks.ModelCheckpoint(
            save_to,
            monitor='val_loss',
            save_best_only=True,
            verbose=1
        ),
        # Stop early if the model overfits
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    ],
    epochs=EPOCHS,
    verbose=1
)

Epoch 1/50
137/137 [==============================] - 216s 631ms/step - loss: 0.6321 - mae: 0.5408 - mape: 289.2203 - val_loss: 0.0299 - val_mae: 0.1394 - val_mape: 27.7185

Epoch 00001: val_loss improved from inf to 0.02988, saving model to ./ckpt_darnn_lstm_logsparse.hdf5
Epoch 2/50
137/137 [==============================] - 53s 389ms/step - loss: 0.0176 - mae: 0.0892 - mape: 34.5479 - val_loss: 0.0123 - val_mae: 0.0949 - val_mape: 14.1987

Epoch 00002: val_loss improved from 0.02988 to 0.01231, saving model to ./ckpt_darnn_lstm_logsparse.hdf5
Epoch 3/50
137/137 [==============================] - 53s 388ms/step - loss: 0.0138 - mae: 0.0805 - mape: 35.8417 - val_loss: 0.0067 - val_mae: 0.0640 - val_mape: 10.8855

Epoch 00003: val_loss improved from 0.01231 to 0.00666, saving model to ./ckpt_darnn_lstm_logsparse.hdf5
Epoch 4/50
137/137 [==============================] - 53s 388ms/step - loss: 0.0053 - mae: 0.0560 - mape: 34.8701 - val_loss: 0.0047 - val_mae: 0.0543 - val_mape: 9.2522



KeyboardInterrupt: ignored

In [ ]:
results = model.evaluate(test_ds)

In [ ]:
preds = model.predict(test_ds)
preds.shape

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
plt.plot(history.history['loss'], label='MSE')
# plt.plot(model.y[model.train_timesteps:], label="True")
plt.legend(loc='upper left')
plt.show()
plt.close(fig)

fig = plt.figure()
plt.plot(preds, label='Predicted')
plt.plot(test_l[:, 0, :], label="True")
plt.legend(loc='upper left')
plt.show()
plt.close(fig)


In [ ]:
model.count_params()

In [ ]:
model.summary()

In [ ]:
import importlib
importlib.reload(keras_lmu)

In [29]:
model_lmu = DARNN(
    T = WINDOW_SIZE,
    m = ENCODER_HIDDEN_STATES,
    d = 4,
    p = DECODER_HIDDEN_STATES,
    y_dim = Y_DIM,
    lmu = True
)

model_lmu.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae', 'mape']
)


In [30]:
feature_batch, label_batch = next(iter(train_ds))

print('feature, label shape', feature_batch.shape, label_batch.shape)

print('prediction shape', model_lmu(feature_batch).shape)

model_lmu(feature_batch[:1])

feature, label shape (256, 50, 82) (256, 1, 1)
prediction shape (256, 1)


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.37261593]], dtype=float32)>

In [32]:
save_to = './ckpt_darnn_lmu.hdf5'

history = model_lmu.fit(
    train_ds,
    validation_data=val_ds,
    callbacks=[
        # Save checkpoints on best validation loss
        tf.keras.callbacks.ModelCheckpoint(
            save_to,
            monitor='val_loss',
            save_best_only=True,
            verbose=1
        ),
        # Stop early if the model overfits
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    ],
    epochs=EPOCHS,
    verbose=1
)

Epoch 1/50
 13/137 [=>............................] - ETA: 35s - loss: 0.0042 - mae: 0.0481 - mape: 3.4486

KeyboardInterrupt: ignored

In [ ]:
results = model_lmu.evaluate(test_ds)

In [ ]:
preds = model_lmu.predict(test_ds)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
plt.plot(history.history['loss'], label='MSE')
# plt.plot(model.y[model.train_timesteps:], label="True")
plt.legend(loc='upper left')
plt.show()
plt.close(fig)

fig = plt.figure()
plt.plot(preds, label='Predicted')
plt.plot(test_l[:, 0, :], label="True")
plt.legend(loc='upper left')
plt.show()
plt.close(fig)